In [ ]:
# dependencies
import numpy as np
from scipy.io import loadmat
from keras.models import Model, load_model
from keras.layers import Input, CuDNNLSTM, Dense
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence

In [ ]:
## making pairs with negative sampling
# generating positive and negative couples using skipgram module from keras
seqs = loadmat('seqs.mat')['list3']
vocab_size = seqs.max() + 1
window_size = 1
sampling_table = sequence.make_sampling_table(vocab_size)
p1 = []
p2 = []
out = []    
for x in seqs:
    couples, labels = sequence.skipgrams(x, vocab_size, window_size=window_size, sampling_table=sampling_table)
    for cpl, lbl in zip(couples, labels):
        p1.append(cpl[0])
        p2.append(cpl[1])
        out.append(lbl)

In [ ]:
## MODEL DESIGN
# model variables
in_dim = vocab_size
red_dim = 600
out_dim = 1
n_hidden = 300

# model components
encoder_inputs = Input(shape = (None, in_dim), name = 'encoder_inputs')
decoder_inputs = Input(shape = (None, in_dim), name = 'decoder_inputs')
pre_dense = Dense(red_dim, activation = 'selu', name = 'pre_dense')
encoder = CuDNNLSTM(n_hidden, return_sequences = True, return_state = True, name = 'encoder')
decoder = CuDNNLSTM(n_hidden, return_sequences = True, name = 'decoder')
dense = Dense(out_dim, activation = 'sigmoid', name = 'dense')

# data flow
pre_dense_out1 = pre_dense(encoder_inputs)
encoder_out, state_h, state_c = encoder(pre_dense_out1)
encoder_states = [state_h, state_c]
pre_dense_out2 = pre_dense(decoder_inputs)
decoder_out = decoder(pre_dense_out2, initial_state = encoder_states)
dense_out = dense(decoder_out)

# constructing the model
training_model = Model([encoder_inputs, decoder_inputs], dense_out)
training_model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])

In [ ]:
## TRAINING
# training dependencies
from keras.utils.np_utils import to_categorical

# converting into numpy arrays
encoder_in = np.asarray(p1, dtype = 'int')
decoder_in = np.asarray(p2, dtype = 'int')
train_labels = np.asarray(out, dtype = 'int')

encoder_in = to_categorical(encoder_in, num_classes = in_dim)
decoder_in = to_categorical(decoder_in, num_classes = in_dim)

# reshaping into 3D LSTM input
encoder_in = np.reshape(encoder_in, (len(encoder_in), 1, in_dim))
decoder_in = np.reshape(decoder_in, (len(decoder_in), 1, in_dim))
train_labels = np.reshape(train_labels, (len(train_labels), 1, out_dim))

# training variables
epochs = 25
batch_size = 1000

# training the data
print 'FORWARD TRAINING STARTED'
training_model.fit([encoder_in, decoder_in], train_labels, epochs = epochs, batch_size = batch_size, verbose = 1)
print 'FORWARD TRAINING FINISHED'

print 'BACKWARD TRAINING STARTED'
training_model.fit([decoder_in, encoder_in], train_labels, epochs = epochs, batch_size = batch_size, verbose = 1)
print 'BACKWARD TRAINING FINISHED'

In [ ]:
## retrieving embeddings
dense_weights = training_model.layers[2].get_weights()
trained_weights = training_model.layers[4].get_weights()
for i in range(len(dense_weights)):
    trained_weights.insert(i, dense_weights[i])

# embedding model components
inputs = Input(shape = (None, in_dim), name = 'inputs')
pre_dense = Dense(red_dim, activation = 'relu', name = 'pre_dense')
lstm_cell = CuDNNLSTM(n_hidden, return_sequences = True, name = 'lstm_cell')
pre_dense_out = pre_dense(inputs)
lstm_out = lstm_cell(pre_dense_out)

# building the model.
pred_model = Model(inputs, lstm_out)
pred_model.set_weights(trained_weights)
pred_model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])

# creating prediction data
pred_data = np.arange(0, in_dim)
pred_data = to_categorical(pred_data, num_classes = in_dim)
pred_data = np.reshape(pred_data, (len(pred_data), 1, in_dim))

# making predictions
preds = pred_model.predict(pred_data)
req_emb = np.reshape(preds, (len(preds), n_hidden))

# saving the embedding as numpy array
np.save('node_emb.npy', req_emb)